In [1]:
import pandas as pd
import numpy as np
import os 
from pathlib import Path

In [2]:
season_code = "24_25"
current_dir = Path.cwd()  
path_folder = current_dir.parent.parent.parent / "csv" / f"csv{season_code}" / "players" / "centiles"
path_folder_end = current_dir.parent.parent.parent / "csv" / f"csv{season_code}" / "players" / "metrics"

In [3]:
def shouting_index(df):
    weights = {
        "Goals": 25,
        "Efficiency": 40,
        "Shots Total": 15,
        "Shots on Target": 20
    }
    
    total_weights = sum(weights.values())
    return df[list(weights.keys())].multiply(pd.Series(weights), axis=1).sum(axis=1) / total_weights

In [4]:
def creation_index(df):
    weights = {
        "Assists": 15,
        "Expected Assists (xA)": 15,
        "Actions created": 30,
        "Through Balls": 20,
        "Actions in the Penalty Area": 20, 
        "Successful Take-Ons": 20
    }
    
    total_weights = sum(weights.values())
    return df[list(weights.keys())].multiply(pd.Series(weights), axis=1).sum(axis=1) / total_weights

In [5]:
def passing_index(df):
    weights = {
        "Passes Completed": 20,
        "Progressive Passes": 20,
        "Passes into Final Third": 20,
        "Key Passes": 20,
        "% Passes (Total)": 20
    }

    total_weights = sum(weights.values())
    return df[list(weights.keys())].multiply(pd.Series(weights), axis=1).sum(axis=1) / total_weights

In [6]:
def possession_index(df):
    weights = {
        "Touches": 25,
        "Carries": 25,
        "Progressive Carries": 25,
        "Carries into Final Third": 20,
        "Touches in Attacking Third": 20
    }

    total_weights = sum(weights.values())
    return df[list(weights.keys())].multiply(pd.Series(weights), axis=1).sum(axis=1) / total_weights

In [7]:
def defense_index(df):
    weights = {
        "Tackles": 20,
        "Tackles Won": 30,
        "Interceptions": 30,
        "Blocks": 10,
        "Clearances": 10,
        "Total Duels Won": 20,
        "Aerials Won": 20, 
        "Ball Recoveries": 30
    }

    total_weights = sum(weights.values())
    return df[list(weights.keys())].multiply(pd.Series(weights), axis=1).sum(axis=1) / total_weights

In [8]:
def global_index(row):
    pos = row['General Position']
        
    if pos == "Defender":
        weights = {
            "Shouting Index": 2,
            "Creation Index": 1,
            "Passing Index": 3,
            "Possession Index": 3,
            "Defense Index": 6
        }
    elif pos == "Midfielder":
        weights = {
            "Shouting Index": 2,
            "Creation Index": 2,
            "Passing Index": 3,
            "Possession Index": 3,
            "Defense Index": 5
        }
    elif pos == "Forward":
        weights = {
            "Shouting Index": 5,
            "Creation Index": 5,
            "Passing Index": 3,
            "Possession Index": 3,
            "Defense Index": 1
        }
    else:
        weights = {
            "Shouting Index": 2,
            "Creation Index": 2,
            "Passing Index": 2,
            "Possession Index": 2,
            "Defense Index": 2
        }
        
    total_weight = sum(weights.values())
    global_index = (
        row["Shouting Index"] * weights["Shouting Index"] +
        row["Creation Index"] * weights["Creation Index"] +
        row["Passing Index"] * weights["Passing Index"] +
        row["Possession Index"] * weights["Possession Index"] +
        row["Defense Index"] * weights["Defense Index"]
    ) / total_weight
        
    return round(global_index)

In [9]:
def create_indices_df(df_centiles):
    static_cols = ['Player', 'Nationality', 'Team', 'League', 'Position', 'General Position', 'Age', 'Minutes', 'Matches']
    
    df_indices = df_centiles[static_cols].copy()
    
    df_indices["Shouting Index"] = shouting_index(df_centiles).round().astype(int)
    df_indices["Creation Index"] = creation_index(df_centiles).round().astype(int)
    df_indices["Passing Index"] = passing_index(df_centiles).round().astype(int)
    df_indices["Possession Index"] = possession_index(df_centiles).round().astype(int)
    df_indices["Defense Index"] = defense_index(df_centiles).round().astype(int)
    
    df_indices["Global Index"] = df_indices.apply(global_index, axis=1)
    
    return df_indices

In [10]:
os.makedirs(path_folder_end, exist_ok=True)

for filename in os.listdir(path_folder):
    if filename.endswith("data_players_centiles.csv"):
        path_file = os.path.join(path_folder, filename)
        
        path_end_indices = path_folder_end / "data_players_metrics.csv"

        df_centiles = pd.read_csv(path_file)
        df_indices = create_indices_df(df_centiles)
        df_indices.to_csv(path_end_indices, index=False)

        print(f"Fichier traité et sauvegardé : {filename}")

Fichier traité et sauvegardé : data_players_centiles.csv
